In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
################################
# Imports
################################

import pandas as pd
import h5py
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import glob
import re
from multiprocessing import cpu_count
from multiprocessing import Pool

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import os
import sys
import subprocess
import random
from random import sample
from tqdm import tqdm
from functools import partial

from sklearn.metrics import confusion_matrix


import librosa

In [16]:
################################
# configs
################################

#################
# spectrogram
#################

# Mean of flute = -6.438913345336914
# Median of flute = -6.118330955505371
# stdDev of flute = 4.377377986907959
# Max of flute = 1.8442230224609375
# Min of flute = -39.0754280090332

# Mean of piano = -6.015857219696045
# Median of piano = -5.299488544464111
# stdDev of piano = 4.420877456665039
# Max of piano = 1.5825170278549194
# Min of piano = -40.520179748535156

spectrogramStats = {
                    'flute': {'mean': -6.438913345336914, 'median': -6.118330955505371, 'stdDev': 4.377377986907959, 'max': 1.8442230224609375, 'min': -39.0754280090332},
                    'piano': {'mean': -6.015857219696045, 'median': -5.299488544464111, 'stdDev': 4.420877456665039, 'max': 1.5825170278549194, 'min': -40.520179748535156}
                    }

# standardizationStyleOptions = normal, logNormal, uniform
# following needs to be 'SUBTRACTED' from the data
centerOffset = {
                'flute': {'normal': spectrogramStats['flute']['mean'], 'logNormal': spectrogramStats['flute']['mean'], 'uniform': (spectrogramStats['flute']['min'] + spectrogramStats['flute']['max'])/2},
                'piano': {'normal': spectrogramStats['piano']['mean'], 'logNormal': spectrogramStats['piano']['mean'], 'uniform': (spectrogramStats['piano']['min'] + spectrogramStats['piano']['max'])/2}
                }

# following needs to be 'DIVIDED' to the data
divFactor = {
            'flute': {'normal': 3*spectrogramStats['flute']['stdDev'], 'logNormal': (1.1*spectrogramStats['flute']['max'] - spectrogramStats['flute']['mean']) , 'uniform': 1*(spectrogramStats['flute']['max'] - spectrogramStats['flute']['min'])/2},
            'piano': {'normal': 3*spectrogramStats['piano']['stdDev'], 'logNormal': (1.1*spectrogramStats['piano']['max'] - spectrogramStats['piano']['mean']) , 'uniform': 1*(spectrogramStats['piano']['max'] - spectrogramStats['piano']['min'])/2}
            }


####################
# training params
####################
STANDARDIZATION_STYLE = 'uniform'
BATCH_SIZE = 1
NUM_WORKERS = 1
NUM_EPOCHS = 6
LEARNING_RATE = 0.0001
LAMBDA_CYCLE = 10
LAMBDA_IDENTITY = 5
GRADIENT_PENALTY = 10
ADAM_BETA1 = 0
ADAM_BETA2 = 0.9
NUM_RESIDUALS = 9

####################
# checkpoint options
####################
SAVE_CHECKPOINTS = True

####################
# find GPU device
####################
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (f'Device = {DEVICE}, # of CUDA devices = {torch.cuda.device_count()}')


###################
# add fileTag
###################
fileTag = f'lr_{LEARNING_RATE}_cyc_{LAMBDA_CYCLE}_id_{LAMBDA_IDENTITY}_b1_{ADAM_BETA1}_b2_{ADAM_BETA2}_numRes_{NUM_RESIDUALS}_gp_{GRADIENT_PENALTY}_stdStyle_{STANDARDIZATION_STYLE}'


Device = cuda:0, # of CUDA devices = 1


In [17]:
################################
# discriminator
################################

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 4, stride = stride, padding = 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=1, features=[64, 128, 256, 512], numFlatFeatures = 1200):
        
        super().__init__()
        
        # output shape = 168 x 128
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size = 4, stride = 2, padding = 1, bias=True, padding_mode="reflect"),            
            nn.LeakyReLU(0.2)
        )
        
        
        layers = []   
        in_channels = features[0]
        
        # linear layer will have 20 x 15 features flattened
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=1 if feature == features[-1] else 2))
            in_channels = feature        
        
        layers.append(nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"), 
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(in_features = numFlatFeatures, out_features = 1)
        ))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):        
        x = self.initial(x)
        return torch.sigmoid(self.model(x))
        
    
def test():
    x = torch.randn((5,1,336,256))
    model = Discriminator(in_channels=1)
    print (model)
    preds = model(x)
    print(preds.shape)

test()

Discriminator(
  (initial): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (model): Sequential(
    (0): Block(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): Block(
      (conv): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (2): Block(
      (conv): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(512, eps=1e-05, momentum

In [18]:
################################
# generator
################################

class EncoderConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_act=True, use_tanh = False, **kwargs):
        super().__init__()
        self.encoderConv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs),
            nn.InstanceNorm2d(out_channels),
            (nn.Tanh() if use_tanh else nn.ReLU(inplace=True)) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.encoderConv(x)


class DecoderConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, use_act=True, use_tanh = False):
        super().__init__()
        
        # nearest neighbor upsample + same convolution
        self.decoderConv = nn.Sequential(            
            nn.Upsample(scale_factor = 2, mode='nearest'),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=0),
            nn.InstanceNorm2d(out_channels),
            (nn.Tanh() if use_tanh else nn.ReLU(inplace=True)) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.decoderConv(x)
    

class ResidualBlock(nn.Module):
    def __init__(self, channels, use_tanh = False):
        super().__init__()
        self.block = nn.Sequential(
            EncoderConvBlock(channels, channels, use_act = True,  use_tanh = use_tanh, kernel_size=3, stride = 1, padding=1),
            EncoderConvBlock(channels, channels, use_act = False, use_tanh = use_tanh, kernel_size=3, stride = 1, padding=1)
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    
    def __init__(self, img_channels, use_tanh = False, num_features = 64, num_residuals=9):
        
        super().__init__()
        
        # same convolution. output channels = 64
        self.initial = nn.Sequential(            
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.ReLU(inplace=True)
        )
        
        
        self.down_blocks = nn.ModuleList(
            [
                # output shape = floor((W - F + 2P) / S) + 1 . 
                # Wout x Hout = 128 x 168, for Win x Hin = 256 x 336, output channels = 128
                EncoderConvBlock(num_features, num_features*2, use_act = True, use_tanh = use_tanh, kernel_size=3, stride=2, padding=1),
                
                # Wout x Hout = 64 x 84, for Win x Hin = 128 x 168, output channels = 256
                EncoderConvBlock(num_features*2, num_features*4, use_act = True, use_tanh = use_tanh, kernel_size=3, stride=2, padding=1),
            ]
        )

        # same convolutions in residual blocks. Shape remains 64 x 86, numChannels = 256
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4, use_tanh = use_tanh) for _ in range(num_residuals)]
        )

        self.up_blocks = nn.ModuleList(
            [
                # Wout x Hout = 128 x 168, numChannels = 128
                DecoderConvBlock(num_features*4, num_features*2, use_act = True, use_tanh = use_tanh),
                
                # Wout x Hout = 256 x 336, numChannels = 64
                DecoderConvBlock(num_features*2, num_features, use_act = True, use_tanh = use_tanh)
            ]
        )

        # same convolution, numChannels = 1
        self.last = nn.Conv2d(num_features, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")

    def forward(self, x):
        
        # same convolution
        x = self.initial(x)
        
        # down sampling
        for layer in self.down_blocks:
            x = layer(x)
            
        # same convolutions
        x = self.residual_blocks(x)
        
        # upsampling
        for layer in self.up_blocks:
            x = layer(x)
        
        # same convolution
        x = self.last(x)
        
        return torch.tanh(x)

    
def test():
    x = torch.randn((5,1,336,256))
    model = Generator(img_channels=1, num_features = 16, num_residuals=9)
    print (model)
    gen = model(x)    
    print(gen.shape)

    
test()

Generator(
  (initial): Sequential(
    (0): Conv2d(1, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
    (1): ReLU(inplace=True)
  )
  (down_blocks): ModuleList(
    (0): EncoderConvBlock(
      (encoderConv): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (1): EncoderConvBlock(
      (encoderConv): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
  )
  (residual_blocks): Sequential(
    (0): ResidualBlock(
      (block): Sequential(
        (0): EncoderConvBlock(
          (encoderConv): Sequential(
            (0): Conv2d(64, 64, kernel_si

torch.Size([5, 1, 336, 256])


In [19]:
# data loader

class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, rootDir):
        'Initialization'
        self.rootDir = rootDir
        self.fileList = os.listdir(rootDir)        
        

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.fileList)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = np.load(os.path.join(self.rootDir, self.fileList[index]))
        
        return X

In [20]:
##########################
# dataset class
##########################

class PianoFluteDataset(Dataset):
    def __init__(self, root_piano, root_flute, standardizationStyle):
        self.root_piano = root_piano
        self.root_flute = root_flute

        random.seed(2)
        self.piano_images = os.listdir(root_piano)
        self.flute_images = sample(os.listdir(root_flute), len(os.listdir(root_flute)))
        
        self.length_dataset = max(len(self.piano_images), len(self.flute_images))

        self.piano_dataset_length = len(self.piano_images)
        self.flute_dataset_length = len(self.flute_images)
        
        self.standardizationStyle = standardizationStyle
        
        # default to normal for standardizing the data
        self.offsetFlute = centerOffset['flute']['normal']
        self.offsetPiano = centerOffset['piano']['normal']
        
        self.divFlute = divFactor['flute']['normal']
        self.divPiano = divFactor['piano']['normal']
        
        if standardizationStyle == 'uniform':
            print ('Using uniform assumption...')
            self.offsetFlute = centerOffset['flute']['uniform']
            self.offsetPiano = centerOffset['piano']['uniform']
        
            self.divFlute = divFactor['flute']['uniform']
            self.divPiano = divFactor['piano']['uniform']
            
        else:
            print ('Using logNormal assumption...')
            self.offsetFlute = centerOffset['flute']['logNormal']
            self.offsetPiano = centerOffset['piano']['logNormal']
        
            self.divFlute = divFactor['flute']['logNormal']
            self.divPiano = divFactor['piano']['logNormal']
        

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        
        # samples for training
        flute_image = self.flute_images[index % self.flute_dataset_length]
        piano_image = self.piano_images[index % self.piano_dataset_length]

        flute_path = os.path.join(self.root_flute, flute_image)
        piano_path = os.path.join(self.root_piano, piano_image)
        
        flute_img = np.load(flute_path)
        piano_img = np.load(piano_path)
        
        # add extra dimension for the single channel
        flute_img = flute_img[None, :]
        piano_img = piano_img[None, :]
        
        # standardize the scale
        flute_img = (flute_img - self.offsetFlute) / self.divFlute
        piano_img = (piano_img - self.offsetPiano) / self.divPiano        
        
        
        
        # samples for interpolation for gradient penalty purposes
        flute_random_sample = self.flute_images[random.randint(0, self.flute_dataset_length - 1)]
        piano_random_sample = self.piano_images[random.randint(0, self.piano_dataset_length - 1)]

        flute_path = os.path.join(self.root_flute, flute_random_sample)
        piano_path = os.path.join(self.root_piano, piano_random_sample)
        
        flute_rnd = np.load(flute_path)
        piano_rnd = np.load(piano_path)
        
        # add extra dimension for the single channel
        flute_rnd = flute_rnd[None, :]
        piano_rnd = piano_rnd[None, :]
        
        # standardize the scale
        flute_rnd = (flute_rnd - self.offsetFlute) / self.divFlute
        piano_rnd = (piano_rnd - self.offsetPiano) / self.divPiano
                        

        return flute_img, piano_img, flute_rnd, piano_rnd

In [21]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):    
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

In [22]:
def gradient_penalty(critic, real, fake, device = 'cpu'):
        
    b, c, h, w = real.shape
    epsilon = torch.rand((b, 1, 1, 1)).repeat(1, c, h, w).to(device)
    interpolated_images = real * epsilon + fake * (1 - epsilon)
    
    mixed_scores = critic(interpolated_images)
    gradient = torch.autograd.grad(
                    inputs = interpolated_images, 
                    outputs = mixed_scores,
                    grad_outputs = torch.ones_like(mixed_scores),
                    create_graph = True,
                    retain_graph = True
                    )[0]
    
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim = 1)
    gradient_penalty = torch.mean((gradient_norm-1)**2)
    
    
    return gradient_penalty


In [23]:
################################
# paths to directories
################################

curDir = os.getcwd()
trainSetDir = f'{curDir}/../../dataSuperSet/processedData/trainSet'
fluteTrainSetDir = f'{trainSetDir}/flute/cqtChunks'
pianoTrainSetDir = f'{trainSetDir}/piano/cqtChunks'

# create directories to store checkpoint outputs
checkPointDir = f'{curDir}/checkPoints'
checkPointModelDir = f'{checkPointDir}/models'
checkPointImageDir = f'{checkPointDir}/images'
checkPointLossTrackingDir = f'{checkPointDir}/lossTracking'

os.system(f'mkdir -p {checkPointDir}')
os.system(f'mkdir -p {checkPointModelDir}')
os.system(f'mkdir -p {checkPointImageDir}')
os.system(f'mkdir -p {checkPointLossTrackingDir}')

0

In [24]:
################################
# datasets
################################

# create dataset
dataset = PianoFluteDataset(pianoTrainSetDir, fluteTrainSetDir, STANDARDIZATION_STYLE)

# create dataloader
loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

Using uniform assumption...


In [25]:
#########################################
# instantiate networks and optimizers
#########################################

# discriminator piano
disc_P = Discriminator(in_channels=1).to(DEVICE)

# discriminator flute
disc_F = Discriminator(in_channels=1).to(DEVICE)

# generator piano
gen_P = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# generator piano
gen_F = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# optimizer discriminator
opt_disc = optim.Adam(list(disc_P.parameters()) + list(disc_F.parameters()), lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))

# optimizer generator
opt_gen  = optim.Adam(list(gen_P.parameters())  + list(gen_F.parameters()),  lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))


# Losses

# For cycle consistency and identity loss
L1 = nn.L1Loss() 

# adversarial loss
mse = nn.MSELoss()


In [26]:
################################
# training
################################

generatorLossProgression = np.zeros(NUM_EPOCHS)
discriminatorLossProgression = np.zeros(NUM_EPOCHS)
identityLossProgression = np.zeros(NUM_EPOCHS)
cycleLossProgression = np.zeros(NUM_EPOCHS)

for epoch in range(NUM_EPOCHS):    
    
    for idx, (flute, piano, flute_rnd, piano_rnd) in enumerate(loader):
        
        # move data to device
        piano = piano.to(DEVICE)
        flute = flute.to(DEVICE)
        
        piano_rnd = piano_rnd.to(DEVICE)
        flute_rnd = flute_rnd.to(DEVICE)
        
        ##############################
        # Discriminator training
        ##############################                       
        
        # piano generator output 
        fake_piano = gen_P (flute)        
        
        # piano discriminator        
        D_P_real = disc_P(piano)
        D_P_fake = disc_P(fake_piano)
        
        # real = 1, fake = 0
        D_P_real_loss = mse(D_P_real, torch.ones_like(D_P_real))
        D_P_fake_loss = mse(D_P_fake, torch.zeros_like(D_P_real))
                          
        # gradient penalty                        
        gp = gradient_penalty(disc_P, piano_rnd, fake_piano, device = DEVICE)
        
        # total piano disc loss
        Disc_piano_loss = D_P_real_loss + D_P_fake_loss + GRADIENT_PENALTY*gp
        
        
        
        # flute generator output
        fake_flute = gen_F(piano)
        
        # flute discriminator
        D_F_real = disc_F(flute)
        D_F_fake = disc_F(fake_flute)
        
        # real = 1, fake = 0
        D_F_real_loss = mse(D_F_real, torch.ones_like(D_F_real))
        D_F_fake_loss = mse(D_F_fake, torch.zeros_like(D_F_real))
        
        # gradient penalty
        gp = gradient_penalty(disc_F, flute_rnd, fake_flute, device = DEVICE)
        
        # total flute disc loss
        Disc_flute_loss = D_F_real_loss + D_F_fake_loss + GRADIENT_PENALTY*gp   
        
        

        # Overall discriminator loss
        D_loss = (Disc_flute_loss + Disc_piano_loss)/2

        # zero out the gradients
        opt_disc.zero_grad()
        
        # backprop
        D_loss.backward(retain_graph = True)
        
        # update discriminator params
        opt_disc.step()        
        
        ##############################
        # Generator training
        ##############################
        
        # Adversarial Loss for both generators
        D_P_fake = disc_P(fake_piano)
        D_F_fake = disc_F(fake_flute)
        
        # generator wants fake to be detected real
        loss_G_F = mse(D_F_fake, torch.ones_like(D_F_fake))
        loss_G_P = mse(D_P_fake, torch.ones_like(D_P_fake))

        # Cycle Loss
        cycle_piano = gen_P(fake_flute)
        cycle_flute = gen_F(fake_piano)
        cycle_piano_loss = L1(piano, cycle_piano)
        cycle_flute_loss = L1(flute, cycle_flute)
        
        cycle_loss = cycle_flute_loss + cycle_piano_loss

        # Identity Loss
        identity_flute = gen_F(flute)
        identity_piano = gen_P(piano)
        identity_piano_loss = L1(piano, identity_piano)
        identity_flute_loss = L1(flute, identity_flute)
        
        identity_loss = identity_flute_loss + identity_piano_loss

        # Overall Generator Loss
        G_loss = loss_G_F + loss_G_P + cycle_loss*LAMBDA_CYCLE + identity_loss*LAMBDA_IDENTITY

        # zero out the gradients
        opt_gen.zero_grad()
        
        # backprop
        G_loss.backward()
        
        # update generator params
        opt_gen.step()
        
        if idx % 400 == 0:
            
            # print the current losses
            print (f'[epoch = {epoch}, idx = {idx}]:   D_loss = {D_loss.item()}, \t G_loss = {G_loss.item()}, \t identity_loss = {identity_loss.item()}, \t cycle_loss = {cycle_loss.item()}')            
            
        if idx % 5000 == 0:      
            
            # save the fake piano and flute np array of a sample            
            np.save(f'{checkPointImageDir}/originalPiano__idx_{idx}__{fileTag}.npy', np.squeeze(piano.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakeFlute__idx_{idx}__{fileTag}.npy', np.squeeze(fake_flute.detach().cpu().numpy()[0,:]))            
            np.save(f'{checkPointImageDir}/originalFlute__idx_{idx}__{fileTag}.npy', np.squeeze(flute.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakePiano__idx_{idx}__{fileTag}.npy', np.squeeze(fake_piano.detach().cpu().numpy()[0,:]))
        
        
        # aggregate loss over all batches
        discriminatorLossProgression[epoch] += D_loss.item()
        generatorLossProgression[epoch] += G_loss.item()
        identityLossProgression[epoch] += identity_loss.item()
        cycleLossProgression[epoch] += cycle_loss.item()
                
                
    
    # average loss over the length of dataset
    discriminatorLossProgression[epoch] = discriminatorLossProgression[epoch] / len(dataset)
    generatorLossProgression[epoch] = generatorLossProgression[epoch] / len(dataset)
    identityLossProgression[epoch] = identityLossProgression[epoch] / len(dataset)
    cycleLossProgression[epoch] = cycleLossProgression[epoch] / len(dataset)
    
    # save the model and current losses
    if SAVE_CHECKPOINTS:
        save_checkpoint(gen_P, opt_gen, filename = f'{checkPointModelDir}/genp__{fileTag}.pth.tar')
        save_checkpoint(gen_F, opt_gen, filename = f'{checkPointModelDir}/genf__{fileTag}.pth.tar')
        save_checkpoint(disc_P, opt_disc, filename = f'{checkPointModelDir}/criticp__{fileTag}.pth.tar')
        save_checkpoint(disc_F, opt_disc, filename = f'{checkPointModelDir}/criticf__{fileTag}.pth.tar')
    
        # save the lossProgressions
        np.save(f'{checkPointLossTrackingDir}/discriminatorLossProgression__{fileTag}.npy', discriminatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/generatorLossProgression__{fileTag}.npy', generatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/identityLossProgression__{fileTag}.npy', identityLossProgression)
        np.save(f'{checkPointLossTrackingDir}/cycleLossProgression__{fileTag}.npy', cycleLossProgression)
    

    




[epoch = 0, idx = 0]:   D_loss = 2.8402328491210938, 	 G_loss = 25.530075073242188, 	 identity_loss = 1.6447548866271973, 	 cycle_loss = 1.6529566049575806
[epoch = 0, idx = 400]:   D_loss = 0.6410345435142517, 	 G_loss = 1.7861212491989136, 	 identity_loss = 0.0739392340183258, 	 cycle_loss = 0.07240309566259384
[epoch = 0, idx = 800]:   D_loss = 0.6736187934875488, 	 G_loss = 1.985235333442688, 	 identity_loss = 0.08259585499763489, 	 cycle_loss = 0.08477181196212769
[epoch = 0, idx = 1200]:   D_loss = 0.5486127138137817, 	 G_loss = 1.4438331127166748, 	 identity_loss = 0.059190377593040466, 	 cycle_loss = 0.06488726288080215
[epoch = 0, idx = 1600]:   D_loss = 0.8908679485321045, 	 G_loss = 2.642169237136841, 	 identity_loss = 0.13734984397888184, 	 cycle_loss = 0.13403427600860596
[epoch = 0, idx = 2000]:   D_loss = 0.6456790566444397, 	 G_loss = 2.8620758056640625, 	 identity_loss = 0.13891375064849854, 	 cycle_loss = 0.1487942337989807
[epoch = 0, idx = 2400]:   D_loss = 0.509324

[epoch = 0, idx = 20400]:   D_loss = 3.0122430324554443, 	 G_loss = 1.1830214262008667, 	 identity_loss = 0.03435884416103363, 	 cycle_loss = 0.054728101938962936
[epoch = 0, idx = 20800]:   D_loss = 0.47990378737449646, 	 G_loss = 2.0387425422668457, 	 identity_loss = 0.09156958013772964, 	 cycle_loss = 0.09620672464370728
[epoch = 0, idx = 21200]:   D_loss = 0.5619277358055115, 	 G_loss = 1.6878292560577393, 	 identity_loss = 0.07379311323165894, 	 cycle_loss = 0.07644487917423248
[epoch = 1, idx = 0]:   D_loss = 0.5741740465164185, 	 G_loss = 1.4286435842514038, 	 identity_loss = 0.05852019786834717, 	 cycle_loss = 0.07386209815740585
[epoch = 1, idx = 400]:   D_loss = 0.47207164764404297, 	 G_loss = 1.3788553476333618, 	 identity_loss = 0.03292930871248245, 	 cycle_loss = 0.04753102734684944
[epoch = 1, idx = 800]:   D_loss = 0.5028615593910217, 	 G_loss = 1.2743276357650757, 	 identity_loss = 0.0318266898393631, 	 cycle_loss = 0.0537203848361969
[epoch = 1, idx = 1200]:   D_loss =

[epoch = 1, idx = 19200]:   D_loss = 0.5147523880004883, 	 G_loss = 1.0619556903839111, 	 identity_loss = 0.03824692964553833, 	 cycle_loss = 0.04679785296320915
[epoch = 1, idx = 19600]:   D_loss = 0.480742871761322, 	 G_loss = 1.4700697660446167, 	 identity_loss = 0.03302440792322159, 	 cycle_loss = 0.06777167320251465
[epoch = 1, idx = 20000]:   D_loss = 0.6488030552864075, 	 G_loss = 1.2033289670944214, 	 identity_loss = 0.03515210002660751, 	 cycle_loss = 0.06428214907646179
[epoch = 1, idx = 20400]:   D_loss = 0.44490841031074524, 	 G_loss = 1.6148523092269897, 	 identity_loss = 0.05258115008473396, 	 cycle_loss = 0.06463932245969772
[epoch = 1, idx = 20800]:   D_loss = 0.4092012941837311, 	 G_loss = 2.0143496990203857, 	 identity_loss = 0.06740473210811615, 	 cycle_loss = 0.0716128796339035
[epoch = 1, idx = 21200]:   D_loss = 0.5282436013221741, 	 G_loss = 0.9462679624557495, 	 identity_loss = 0.022981632500886917, 	 cycle_loss = 0.036706067621707916
[epoch = 2, idx = 0]:   D_l

[epoch = 2, idx = 18000]:   D_loss = 0.4386780261993408, 	 G_loss = 1.2588481903076172, 	 identity_loss = 0.036617621779441833, 	 cycle_loss = 0.043211646378040314
[epoch = 2, idx = 18400]:   D_loss = 1.1335467100143433, 	 G_loss = 0.9403289556503296, 	 identity_loss = 0.0390763133764267, 	 cycle_loss = 0.0418730303645134
[epoch = 2, idx = 18800]:   D_loss = 0.42367517948150635, 	 G_loss = 1.3240350484848022, 	 identity_loss = 0.037197113037109375, 	 cycle_loss = 0.05175347998738289
[epoch = 2, idx = 19200]:   D_loss = 0.4779144525527954, 	 G_loss = 1.4260029792785645, 	 identity_loss = 0.03319331258535385, 	 cycle_loss = 0.06106434762477875
[epoch = 2, idx = 19600]:   D_loss = 0.5003498792648315, 	 G_loss = 1.1622941493988037, 	 identity_loss = 0.02691233716905117, 	 cycle_loss = 0.039613161236047745
[epoch = 2, idx = 20000]:   D_loss = 0.5019424557685852, 	 G_loss = 1.3271833658218384, 	 identity_loss = 0.03158922493457794, 	 cycle_loss = 0.04700477421283722
[epoch = 2, idx = 20400]:

[epoch = 3, idx = 16800]:   D_loss = 0.5156246423721313, 	 G_loss = 1.0065664052963257, 	 identity_loss = 0.03049841709434986, 	 cycle_loss = 0.04091869667172432
[epoch = 3, idx = 17200]:   D_loss = 0.5486987233161926, 	 G_loss = 1.0411036014556885, 	 identity_loss = 0.024800177663564682, 	 cycle_loss = 0.03468839079141617
[epoch = 3, idx = 17600]:   D_loss = 0.4198361039161682, 	 G_loss = 1.0675760507583618, 	 identity_loss = 0.016473108902573586, 	 cycle_loss = 0.033412497490644455
[epoch = 3, idx = 18000]:   D_loss = 0.5852441787719727, 	 G_loss = 0.8881850242614746, 	 identity_loss = 0.02426673099398613, 	 cycle_loss = 0.036345914006233215
[epoch = 3, idx = 18400]:   D_loss = 0.43217355012893677, 	 G_loss = 1.1548255681991577, 	 identity_loss = 0.02115211822092533, 	 cycle_loss = 0.04409048706293106
[epoch = 3, idx = 18800]:   D_loss = 1.2710583209991455, 	 G_loss = 1.0718966722488403, 	 identity_loss = 0.026825297623872757, 	 cycle_loss = 0.045970283448696136
[epoch = 3, idx = 192

[epoch = 4, idx = 15600]:   D_loss = 0.7677164077758789, 	 G_loss = 0.7904571294784546, 	 identity_loss = 0.01806989684700966, 	 cycle_loss = 0.03907797858119011
[epoch = 4, idx = 16000]:   D_loss = 0.4770315885543823, 	 G_loss = 1.1308248043060303, 	 identity_loss = 0.025812819600105286, 	 cycle_loss = 0.03337007761001587
[epoch = 4, idx = 16400]:   D_loss = 0.5505826473236084, 	 G_loss = 1.02887761592865, 	 identity_loss = 0.020786959677934647, 	 cycle_loss = 0.03283819556236267
[epoch = 4, idx = 16800]:   D_loss = 0.7206847667694092, 	 G_loss = 1.144757866859436, 	 identity_loss = 0.03079722262918949, 	 cycle_loss = 0.04066016525030136
[epoch = 4, idx = 17200]:   D_loss = 0.570199728012085, 	 G_loss = 0.9183504581451416, 	 identity_loss = 0.022929824888706207, 	 cycle_loss = 0.030282240360975266
[epoch = 4, idx = 17600]:   D_loss = 0.5339784622192383, 	 G_loss = 1.2737538814544678, 	 identity_loss = 0.0430682897567749, 	 cycle_loss = 0.042362526059150696
[epoch = 4, idx = 18000]:   

[epoch = 5, idx = 14400]:   D_loss = 0.3668184280395508, 	 G_loss = 1.438385009765625, 	 identity_loss = 0.029394330456852913, 	 cycle_loss = 0.04238024353981018
[epoch = 5, idx = 14800]:   D_loss = 0.6035732626914978, 	 G_loss = 1.6428803205490112, 	 identity_loss = 0.03534357622265816, 	 cycle_loss = 0.04109320044517517
[epoch = 5, idx = 15200]:   D_loss = 0.5037583112716675, 	 G_loss = 1.0559344291687012, 	 identity_loss = 0.022628184407949448, 	 cycle_loss = 0.03536828234791756
[epoch = 5, idx = 15600]:   D_loss = 0.5397672653198242, 	 G_loss = 0.9071933031082153, 	 identity_loss = 0.018680769950151443, 	 cycle_loss = 0.03093056008219719
[epoch = 5, idx = 16000]:   D_loss = 0.5186105370521545, 	 G_loss = 0.9297454953193665, 	 identity_loss = 0.017529789358377457, 	 cycle_loss = 0.034540094435214996
[epoch = 5, idx = 16400]:   D_loss = 0.48306503891944885, 	 G_loss = 1.188388705253601, 	 identity_loss = 0.029064495116472244, 	 cycle_loss = 0.04702705517411232
[epoch = 5, idx = 16800